In [1]:
import gensim
import os
import collections
import smart_open
import random
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import numpy as np
from sklearn import svm
from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
import time

/home/hao/anaconda3/envs/MLOntology/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#global variabls

directory_path =  "/home/hao/AnacondaProjects/MLOntology/NCIt/"
data_path = directory_path + "data/"
vector_model_path = directory_path +"vectorModel/"
cnn_model_path = directory_path +"cnnModel/"

In [3]:
import re
def get_trailing_number(s):
    m = re.search(r'\d+$', s)
    return m.group() if m else None

In [4]:
#read class label file
#create mapping from id to labels  
#iso-8859-1 , encoding="iso-8859-1"
conceptLabelDict={}
errors=[]

def read_label(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                conceptID = get_trailing_number(splitted[1])
                conceptLabelDict[conceptID] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

label_file = data_path + "ontClassLabels_owl_ncit.txt"
read_label(label_file)
print(conceptLabelDict["4863"])
print(conceptLabelDict["115117"])
print(errors)

prostate carcinoma
stage ia esophageal cancer ajcc v7
[]


In [14]:
conceptPairDict={}
errors=[]
conceptPairList=[]

def read_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==3:
                childID = get_trailing_number(splitted[1])
                parentID = get_trailing_number(splitted[2].replace("\r\n", ""))
                conceptPairList.append([childID, parentID , 1])
#                 conceptPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

pair_file = data_path + "ontHierarchy_owl_ncit.txt"
read_pair(pair_file)

checkpairs = conceptPairList[10:15]
print(checkpairs)
print(len(conceptPairList))

[['4861', '7318', 1], ['87152', '87150', 1], ['87153', '140032', 1], ['87154', '87153', 1], ['87155', '87153', 1]]
16533


In [15]:
conceptNotPairDict={}
conceptNotPairList=[]

def read_not_pair(fname):
    with smart_open.smart_open(fname) as f:
        for i, line in enumerate(f):
            #get the id for each concept paragraph
            splitted = line.decode("iso-8859-1").split("\t")
            if len(splitted)==2:
                childID = get_trailing_number(splitted[0])
                notparentID = get_trailing_number(splitted[1].replace("\r\n", ""))
                conceptNotPairList.append([childID, notparentID, 0])
#                 conceptNotPairDict[splitted[1]] = splitted[2].replace("\r\n", "")
            else:
                errors.append(splitted)

notPair_file = data_path + "taxNotPairs_owl_ncit.txt"
read_not_pair(notPair_file)


first2pairs =conceptNotPairList[10:15]
print(first2pairs)
print(len(conceptNotPairList))

[['7918', '9151', 0], ['7918', '48612', 0], ['7918', '48613', 0], ['7918', '91231', 0], ['7918', '66753', 0]]
37147


In [20]:

cleanlist = []
[cleanlist.append(x) for x in conceptNotPairList if x not in cleanlist]
print(len(cleanlist))


34927


In [21]:
conceptNotPairList = cleanlist

In [22]:
import random
new_list = random.sample(conceptPairList, 2000)
print(new_list[2:10])
print(len(new_list))

new_list_2 = random.sample(conceptNotPairList, 2000)
print(new_list_2[2:10])
print(len(new_list_2))

[['8176', '8174', 1], ['6681', '5647', 1], ['54294', '54293', 1], ['7532', '7530', 1], ['4295', '7201', 1], ['3757', '4699', 1], ['140007', '140005', 1], ['95546', '4374', 1]]
2000
[['9061', '4092', 0], ['6106', '4110', 0], ['27822', '5091', 0], ['122717', '3182', 0], ['128112', '3443', 0], ['4572', '2877', 0], ['4647', '5212', 0], ['4381', '4858', 0]]
2000


In [23]:
testing_list = [x for x in conceptPairList if x not in new_list]
testing_list_2 = [y for y in conceptNotPairList if y not in new_list_2]

print(len(testing_list))
print(len(testing_list_2))

14533
32927


In [30]:
test_list = new_list + new_list_2

filepath = directory_path + "output.txt"
with open(filepath, 'w') as file_handler:
    for item in test_list:
        file_handler.write("{}\t{}\t{}\n".format(item[0], item[1], item[2]))

In [36]:
with open(filepath) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content] 

pair_lists=[]
for item in content:
    itemlist=item.split('\t')
    pair_lists.append(itemlist)

print(len(pair_lists))
print(pair_lists[:5])

4000
[['3642', '5541', '1'], ['7546', '7545', '1'], ['8176', '8174', '1'], ['6681', '5647', '1'], ['54294', '54293', '1']]
